In [56]:
#!pip install memory_profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.59.0-py3-none-any.whl size=31306 sha256=160b8853ac47683b9587fdb79de3ea5f633152f3a2ae6536973c6587037f11f6
  Stored in directory: /home/marco/.cache/pip/wheels/5d/66/23/1e7f1719b959ee9093d5025dbdcbe4c43a548ca510997f318f
Successfully built memory-profiler


In [57]:
%load_ext memory_profiler

In [59]:
import time

In [1]:
sc.stop()
from pyspark.sql import SparkSession
from pyspark import SparkContext

SpSession = SparkSession\
    .builder\
    .master("local[4]")\
    .appName("KMeans")\
    .getOrCreate()

sc = SpSession.sparkContext

sc._conf.get('spark.driver.memory')

sc._conf.getAll()

[('spark.master', 'local[4]'),
 ('spark.local.dir', '/home/marco/claseBigData/ProyectoBD/tmp'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.memory', '16g'),
 ('spark.driver.host', '192.168.3.5'),
 ('spark.app.startTime', '1639596503552'),
 ('spark.app.id', 'local-1639596503629'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.name', 'KMeans'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.port', '45001'),
 ('spark.ui.showConsoleProgress', 'true')]

In [2]:
import pandas as pd
from pyspark.ml.linalg import Vectors
from pyspark.sql import Row
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

In [3]:
allDf = SpSession.read.parquet("../prepData/variables_finales.parquet")
# print(allDf.dtypes)
allDf.limit(5).toPandas().head()

,origen,contactos,mt_sumSeg,mt_sumSegMobile,mt_sumSegService,mt_sumSegInternational,mt_sumSegOthers,mt_sumLla,mt_sumLlaMobile,mt_sumLlaService,...,mo_PropSegService,mo_PropSegInternational,mo_PropLlaMobile,mo_PropLlaService,mo_PropLlaInternational,mo_PropNroDestMobile,mo_PropNroDestService,mo_PropNroDestInternational,PropSegMobileTvsTotal,PropContactosTvsTotal
0,0001ca95045d5e5c9768418769b29dfd32bb9d2b7139ad...,2,84.0,84.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.16667,0.16667,0.75,0.25,0.25,0.75,0.25,0.25,0.97126,0.83333
1,0001f53140577468061aa5ca98fa6c366c6bca523fbed8...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.50000,0.50000,0.50,0.50,0.50,0.50,0.50,0.50,0.50000,0.75000
2,00180793f4b84bbb131687aa84e2a1e4e7fb61a6526726...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.98387,0.01613,0.25,0.75,0.25,0.25,0.75,0.25,0.01613,0.25000
3,0019beda12496c067808fa08397f997cde6dcf447658ce...,7,81.0,81.0,0.0,0.0,0.0,3.0,3.0,0.0,...,0.50000,0.50000,0.50,0.50,0.50,0.50,0.50,0.50,0.99390,0.93750
4,001ab61b6509832d04d3638afe98af6675f09d3d265280...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.50000,0.50000,0.50,0.50,0.50,0.50,0.50,0.50,0.50000,0.75000


In [4]:
allDf = allDf.drop('origen')
allDf.limit(5).show(truncate=False)

+---------+---------+---------------+----------------+----------------------+---------------+---------+---------------+----------------+----------------------+---------------+---------+---------------+----------------+----------------------+---------------+----------+----------------+-----------------+-----------------------+----------------+----------------+-----------------+-----------------------+----------------+-----------------+-----------------------+----------------+-----------------+-----------------------+--------------------+---------------------+---------------------------+---------+---------------+----------------+----------------------+---------------+---------+---------------+----------------+----------------------+---------------+----------+----------------+-----------------+-----------------------+----------------+----------------+-----------------+-----------------------+----------------+-----------------+-----------------------+--------------------+------------------

### Calculamos la matriz de correlaciones

In [5]:
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=allDf.columns, outputCol=vector_col)
df_vector = assembler.transform(allDf).select(vector_col)
matrix = Correlation.corr(df_vector, vector_col)

In [6]:
matrix.sql_ctx.sparkSession._jsparkSession = SpSession._jsparkSession
matrix._sc = SpSession._sc
matrix.collect()[0][matrix.columns[0]].toArray()

array([[ 1.        ,  0.27815427,  0.26970191, ..., -0.51054114,
        -0.13592374, -0.05250227],
       [ 0.27815427,  1.        ,  0.98677563, ..., -0.06840788,
         0.26913575,  0.17528191],
       [ 0.26970191,  0.98677563,  1.        , ..., -0.07108977,
         0.26035927,  0.17045259],
       ...,
       [-0.51054114, -0.06840788, -0.07108977, ...,  1.        ,
         0.60676584,  0.55113662],
       [-0.13592374,  0.26913575,  0.26035927, ...,  0.60676584,
         1.        ,  0.67553886],
       [-0.05250227,  0.17528191,  0.17045259, ...,  0.55113662,
         0.67553886,  1.        ]])

In [7]:
cor_np = matrix.collect()[0][matrix.columns[0]].toArray()
cor_np

array([[ 1.        ,  0.27815427,  0.26970191, ..., -0.51054114,
        -0.13592374, -0.05250227],
       [ 0.27815427,  1.        ,  0.98677563, ..., -0.06840788,
         0.26913575,  0.17528191],
       [ 0.26970191,  0.98677563,  1.        , ..., -0.07108977,
         0.26035927,  0.17045259],
       ...,
       [-0.51054114, -0.06840788, -0.07108977, ...,  1.        ,
         0.60676584,  0.55113662],
       [-0.13592374,  0.26913575,  0.26035927, ...,  0.60676584,
         1.        ,  0.67553886],
       [-0.05250227,  0.17528191,  0.17045259, ...,  0.55113662,
         0.67553886,  1.        ]])

In [8]:
# Me vuelo las columnas con sumas totales
high_corr_columns = ['mt_sumSeg', 'mt_sumLla', 'mt_sumSms', 'mt_nroDest', 'mo_sumSeg', 'mo_sumLla', 'mo_sumSms', 'mo_nroDest']

allDf = allDf.drop(*high_corr_columns)
allDf.limit(5).toPandas().head()

,contactos,mt_sumSegMobile,mt_sumSegService,mt_sumSegInternational,mt_sumSegOthers,mt_sumLlaMobile,mt_sumLlaService,mt_sumLlaInternational,mt_sumLlaOthers,mt_sumSmsMobile,...,mo_PropSegService,mo_PropSegInternational,mo_PropLlaMobile,mo_PropLlaService,mo_PropLlaInternational,mo_PropNroDestMobile,mo_PropNroDestService,mo_PropNroDestInternational,PropSegMobileTvsTotal,PropContactosTvsTotal
0,2,84.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.16667,0.16667,0.75,0.25,0.25,0.75,0.25,0.25,0.97126,0.83333
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.50000,0.50000,0.50,0.50,0.50,0.50,0.50,0.50,0.50000,0.75000
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.98387,0.01613,0.25,0.75,0.25,0.25,0.75,0.25,0.01613,0.25000
3,7,81.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,54.0,...,0.50000,0.50000,0.50,0.50,0.50,0.50,0.50,0.50,0.99390,0.93750
4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.50000,0.50000,0.50,0.50,0.50,0.50,0.50,0.50,0.50000,0.75000


In [9]:
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=allDf.columns, outputCol=vector_col)
df_vector = assembler.transform(allDf).select(vector_col)
matrix = Correlation.corr(df_vector, vector_col)
matrix.sql_ctx.sparkSession._jsparkSession = SpSession._jsparkSession
matrix._sc = SpSession._sc
cor_np = matrix.collect()[0][matrix.columns[0]].toArray()
cor_np

array([[ 1.00000000e+00,  2.69701913e-01,  1.40313835e-03, ...,
        -5.10541142e-01, -1.35923736e-01, -5.25022653e-02],
       [ 2.69701913e-01,  1.00000000e+00,  6.17009143e-04, ...,
        -7.10897670e-02,  2.60359265e-01,  1.70452588e-01],
       [ 1.40313835e-03,  6.17009143e-04,  1.00000000e+00, ...,
         1.14486600e-03,  2.93718898e-03,  1.89711225e-03],
       ...,
       [-5.10541142e-01, -7.10897670e-02,  1.14486600e-03, ...,
         1.00000000e+00,  6.06765844e-01,  5.51136620e-01],
       [-1.35923736e-01,  2.60359265e-01,  2.93718898e-03, ...,
         6.06765844e-01,  1.00000000e+00,  6.75538865e-01],
       [-5.25022653e-02,  1.70452588e-01,  1.89711225e-03, ...,
         5.51136620e-01,  6.75538865e-01,  1.00000000e+00]])

In [10]:
tempDf = pd.DataFrame(cor_np, index=allDf.columns,  columns=allDf.columns)
# tempDf.to_csv('./tempDf.csv')

In [11]:
high_corr_columns = ['mt_sumSmsService', 'mt_PropSegMobile', 'mt_PropSegService', 'mt_PropSegInternational', 'mt_PropLlaMobile', 'mt_PropLlaService', 'mt_PropLlaInternational', 'mt_PropSmsService', 'mt_PropNroDestMobile', 'mt_PropNroDestService', 'mo_PropSegMobile', 'mo_PropSegService', 'mo_PropSegInternational', 'mo_PropLlaMobile', 'mo_PropLlaService', 'mo_PropLlaInternational', 'mo_PropNroDestMobile', 'mo_PropNroDestService', 'mo_PropNroDestInternational']
allDf = allDf.drop(*high_corr_columns)
allDf.limit(5).toPandas().head()

,contactos,mt_sumSegMobile,mt_sumSegService,mt_sumSegInternational,mt_sumSegOthers,mt_sumLlaMobile,mt_sumLlaService,mt_sumLlaInternational,mt_sumLlaOthers,mt_sumSmsMobile,...,mo_sumLlaMobile,mo_sumLlaService,mo_sumLlaInternational,mo_sumLlaOthers,mo_nroDestMobile,mo_nroDestService,mo_nroDestInternational,mo_nroDestOthers,PropSegMobileTvsTotal,PropContactosTvsTotal
0,2,84.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1,0,0,0,0.97126,0.83333
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.50000,0.75000
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0,1,0,0,0.01613,0.25000
3,7,81.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,54.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.99390,0.93750
4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0.50000,0.75000


In [12]:
allDf.coalesce(5).write.format("parquet").mode("overwrite").save("../prepData/var_selection.parquet")

In [13]:
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=allDf.columns, outputCol=vector_col)
df_vector = assembler.transform(allDf).select(vector_col)
matrix = Correlation.corr(df_vector, vector_col)
matrix.sql_ctx.sparkSession._jsparkSession = SpSession._jsparkSession
matrix._sc = SpSession._sc
cor_np = matrix.collect()[0][matrix.columns[0]].toArray()
cor_np

array([[ 1.00000000e+00,  2.69701913e-01,  1.40313835e-03, ...,
         2.00739646e-01, -1.35923736e-01, -5.25022653e-02],
       [ 2.69701913e-01,  1.00000000e+00,  6.17009143e-04, ...,
         2.80482345e-02,  2.60359265e-01,  1.70452588e-01],
       [ 1.40313835e-03,  6.17009143e-04,  1.00000000e+00, ...,
        -2.41064629e-04,  2.93718898e-03,  1.89711225e-03],
       ...,
       [ 2.00739646e-01,  2.80482345e-02, -2.41064629e-04, ...,
         1.00000000e+00, -1.18932397e-01, -1.64540009e-01],
       [-1.35923736e-01,  2.60359265e-01,  2.93718898e-03, ...,
        -1.18932397e-01,  1.00000000e+00,  6.75538865e-01],
       [-5.25022653e-02,  1.70452588e-01,  1.89711225e-03, ...,
        -1.64540009e-01,  6.75538865e-01,  1.00000000e+00]])

In [14]:
tempDf = pd.DataFrame(cor_np, index=allDf.columns,  columns=allDf.columns)
# tempDf.to_csv('./tempDf.csv')

### Centrado y escalado

In [15]:
summStats = allDf.describe().toPandas()
# print(summStats)
meanValues = summStats.iloc[1, 1:].values.tolist()
stdValues = summStats.iloc[2, 1:].values.tolist()

# # Colocamos estos valores en una variable de difusion
bcMeans = SpSession.sparkContext.broadcast(meanValues)
bcStdDev = SpSession.sparkContext.broadcast(stdValues)

In [16]:
def normalize(inRow):
    global bcMeans
    global bcStdDev
    
    meanArray = bcMeans.value
    stdArray = bcStdDev.value
    
    retArray = []
    for i in range(len(meanArray)):
        retArray.append((float(inRow[i])-float(meanArray[i]))/float(stdArray[i]))
        
    return Vectors.dense(retArray)

In [17]:
allVectors = allDf.rdd.map(normalize)
allVectors.collect()

[DenseVector([-0.6112, -0.3368, -0.0034, -0.0788, -0.1122, -0.4919, -0.0043, -0.1072, -0.1891, -0.2345, -0.0364, -0.046, -0.2274, -0.8532, -0.1457, -0.2389, -0.1528, 0.0513, -0.0425, -0.3884, -0.2367, -0.0656, -0.099, -0.3883, -0.2931, -0.0821, -0.1509, -0.3061, -0.4185, -0.1065, -0.1896, 1.0328, 0.7056]),
 DenseVector([-0.8258, -0.4264, -0.0034, -0.0788, -0.1122, -0.6936, -0.0043, -0.1072, -0.1891, -0.2345, -0.0364, -0.046, -0.5242, -0.8532, -0.1457, -0.2389, -0.0822, 0.0513, 0.7102, -0.3909, -0.2367, -0.0656, -0.099, -0.5359, -0.2931, -0.0821, -0.1509, -0.6458, -0.4185, -0.1065, -0.1896, -0.2195, 0.2981]),
 DenseVector([-0.8258, -0.4264, -0.0034, -0.0788, -0.1122, -0.6936, -0.0043, -0.1072, -0.1891, -0.274, -0.0364, -0.046, -0.821, -0.8532, -0.1457, -0.2389, -0.1763, 1.4122, 2.9685, -0.3909, 0.3069, -0.0656, -0.099, -0.5359, 0.5011, -0.0821, -0.1509, -0.6458, 1.7847, -0.1065, -0.1896, -1.5053, -2.1468]),
 DenseVector([0.4617, -0.34, -0.0034, -0.0788, -0.1122, -0.0884, -0.0043, -0.107

In [18]:
# Crear un marco de datos de Spark
allRows = allVectors.map(lambda f:Row(features=f))
allDf = SpSession.createDataFrame(allRows)

allDf.select("features").show(10, truncate = False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                                                  

### Evaluando número de cluster a identificar

In [19]:
# from pyspark.ml.feature import StandardScaler

# scale=StandardScaler(inputCol='features', outputCol='standardized')
# data_scale=scale.fit(allVectors)
# data_scale_output=data_scale.transform(allVectors)
# data_scale_output.show(2)

In [20]:
# from pyspark.ml.clustering import KMeans
# from pyspark.ml.evaluation import ClusteringEvaluator

# silhouette_score = []
# evaluator = ClusteringEvaluator(
#     predictionCol='prediction',
#     featuresCol='features',
#     metricName='silhouette',
#     distanceMeasure='squaredEuclidean')

# for i in range(2,16):
#     KMeans_algo=KMeans(featuresCol='features', k=i)
#     KMeans_fit=KMeans_algo.fit(allDf)
#     output=KMeans_fit.transform(allDf)
#     score=evaluator.evaluate(output)
#     silhouette_score.append(score)

#     print("Silhouette Score:",score)

In [22]:
#Visualizing the silhouette scores in a plot
# import matplotlib.pyplot as plt
# fig, ax = plt.subplots(1,1, figsize =(8,6))
# ax.plot(range(2,16),silhouette_score)
# ax.set_xlabel('k')
# ax.set_ylabel('cost')

### Modelo

In [23]:
from pyspark.ml.clustering import KMeans

Kmeans = KMeans(k=5, seed=1)

In [60]:
%%time
%memit model = Kmeans.fit(allDf)

peak memory: 2346.32 MiB, increment: 0.00 MiB
CPU times: user 570 ms, sys: 125 ms, total: 695 ms
Wall time: 5min 7s


In [61]:
%%time
%memit predictions = model.transform(allDf)

peak memory: 2343.59 MiB, increment: 0.00 MiB
CPU times: user 536 ms, sys: 156 ms, total: 692 ms
Wall time: 888 ms


In [24]:

predictions.show(10, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|features                                                                                                                                                                                                                                                                                                       

In [25]:
type(predictions)

pyspark.sql.dataframe.DataFrame

In [26]:
predictions.printSchema()

root
 |-- features: vector (nullable = true)
 |-- prediction: integer (nullable = false)



In [27]:
predictions.count()

3761712

In [28]:
allDf.show()

+--------------------+
|            features|
+--------------------+
|[-0.6112321571936...|
|[-0.8258143728411...|
|[-0.8258143728411...|
|[0.46167892104351...|
|[-0.8258143728411...|
|[-0.8258143728411...|
|[-0.8258143728411...|
|[-0.3966499415462...|
|[-0.8258143728411...|
|[-0.6112321571936...|
|[0.24709670539607...|
|[0.24709670539607...|
|[-0.6112321571936...|
|[0.89084335233840...|
|[0.03251448974863...|
|[-0.3966499415462...|
|[0.89084335233840...|
|[0.89084335233840...|
|[-0.1820677258988...|
|[-0.8258143728411...|
+--------------------+
only showing top 20 rows



In [29]:
# predictions.coalesce(5).write.format("parquet").mode("overwrite").save("../prepData/predicctions.parquet")

In [30]:
predictions.select("features").show()

+--------------------+
|            features|
+--------------------+
|[-0.6112321571936...|
|[-0.8258143728411...|
|[-0.8258143728411...|
|[0.46167892104351...|
|[-0.8258143728411...|
|[-0.8258143728411...|
|[-0.8258143728411...|
|[-0.3966499415462...|
|[-0.8258143728411...|
|[-0.6112321571936...|
|[0.24709670539607...|
|[0.24709670539607...|
|[-0.6112321571936...|
|[0.89084335233840...|
|[0.03251448974863...|
|[-0.3966499415462...|
|[0.89084335233840...|
|[0.89084335233840...|
|[-0.1820677258988...|
|[-0.8258143728411...|
+--------------------+
only showing top 20 rows



In [31]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, DoubleType

def to_array(col):
    def to_array_(v):
        return v.toArray().tolist()
    # Important: asNondeterministic requires Spark 2.3 or later
    # It can be safely removed i.e.
    # return udf(to_array_, ArrayType(DoubleType()))(col)
    # but at the cost of decreased performance
    return udf(to_array_, ArrayType(DoubleType())).asNondeterministic()(col)

In [32]:
predictions.withColumn("contactos", to_array(predictions["features"]).getItem(0)).show()

+--------------------+----------+--------------------+
|            features|prediction|           contactos|
+--------------------+----------+--------------------+
|[-0.6112321571936...|         1| -0.6112321571936897|
|[-0.8258143728411...|         1| -0.8258143728411312|
|[-0.8258143728411...|         3| -0.8258143728411312|
|[0.46167892104351...|         1|  0.4616789210435177|
|[-0.8258143728411...|         1| -0.8258143728411312|
|[-0.8258143728411...|         1| -0.8258143728411312|
|[-0.8258143728411...|         1| -0.8258143728411312|
|[-0.3966499415462...|         1| -0.3966499415462483|
|[-0.8258143728411...|         1| -0.8258143728411312|
|[-0.6112321571936...|         1| -0.6112321571936897|
|[0.24709670539607...|         1| 0.24709670539607623|
|[0.24709670539607...|         3| 0.24709670539607623|
|[-0.6112321571936...|         1| -0.6112321571936897|
|[0.89084335233840...|         0|  0.8908433523384007|
|[0.03251448974863...|         3| 0.03251448974863473|
|[-0.39664

In [33]:
dfVarSelecction = SpSession.read.parquet("../prepData/var_selection.parquet")

In [34]:
predictionsDf = dfVarSelecction.join(predictions.select("prediction"))

In [35]:
predictionsDf.limit(10).toPandas().head()

,contactos,mt_sumSegMobile,mt_sumSegService,mt_sumSegInternational,mt_sumSegOthers,mt_sumLlaMobile,mt_sumLlaService,mt_sumLlaInternational,mt_sumLlaOthers,mt_sumSmsMobile,...,mo_sumLlaService,mo_sumLlaInternational,mo_sumLlaOthers,mo_nroDestMobile,mo_nroDestService,mo_nroDestInternational,mo_nroDestOthers,PropSegMobileTvsTotal,PropContactosTvsTotal,prediction
0,2,84.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1,0,0,0,0.97126,0.83333,1
1,2,84.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1,0,0,0,0.97126,0.83333,1
2,2,84.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1,0,0,0,0.97126,0.83333,3
3,2,84.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1,0,0,0,0.97126,0.83333,1
4,2,84.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1,0,0,0,0.97126,0.83333,1


In [36]:
from pyspark.sql.functions import monotonically_increasing_id
dfVarSelecction = dfVarSelecction.withColumn("code", monotonically_increasing_id())
predDf = predictions.select("prediction").withColumn("code", monotonically_increasing_id())
dfVarAndPred = dfVarSelecction.join(predDf, ["code"]).drop("code")

In [37]:
dfVarAndPred.show()

+---------+---------------+----------------+----------------------+---------------+---------------+----------------+----------------------+---------------+---------------+----------------------+---------------+----------------+-----------------+-----------------------+----------------+----------------+-----------------------+---------------------------+---------------+----------------+----------------------+---------------+---------------+----------------+----------------------+---------------+----------------+-----------------+-----------------------+----------------+---------------------+---------------------+----------+
|contactos|mt_sumSegMobile|mt_sumSegService|mt_sumSegInternational|mt_sumSegOthers|mt_sumLlaMobile|mt_sumLlaService|mt_sumLlaInternational|mt_sumLlaOthers|mt_sumSmsMobile|mt_sumSmsInternational|mt_sumSmsOthers|mt_nroDestMobile|mt_nroDestService|mt_nroDestInternational|mt_nroDestOthers|mt_PropSmsMobile|mt_PropSmsInternational|mt_PropNroDestInternational|mo_sumSegMobil

In [38]:
dfVarAndPred.count()

3761712

In [39]:
dfVarAndPred.coalesce(5).write.format("parquet").mode("overwrite").save("../prepData/Vars33OriAndPred.parquet")

In [40]:
dfVarAndPred.printSchema()

root
 |-- contactos: long (nullable = true)
 |-- mt_sumSegMobile: double (nullable = true)
 |-- mt_sumSegService: double (nullable = true)
 |-- mt_sumSegInternational: double (nullable = true)
 |-- mt_sumSegOthers: double (nullable = true)
 |-- mt_sumLlaMobile: double (nullable = true)
 |-- mt_sumLlaService: double (nullable = true)
 |-- mt_sumLlaInternational: double (nullable = true)
 |-- mt_sumLlaOthers: double (nullable = true)
 |-- mt_sumSmsMobile: double (nullable = true)
 |-- mt_sumSmsInternational: double (nullable = true)
 |-- mt_sumSmsOthers: double (nullable = true)
 |-- mt_nroDestMobile: long (nullable = true)
 |-- mt_nroDestService: long (nullable = true)
 |-- mt_nroDestInternational: long (nullable = true)
 |-- mt_nroDestOthers: long (nullable = true)
 |-- mt_PropSmsMobile: double (nullable = true)
 |-- mt_PropSmsInternational: double (nullable = true)
 |-- mt_PropNroDestInternational: double (nullable = true)
 |-- mo_sumSegMobile: double (nullable = true)
 |-- mo_sumSegS

In [45]:
from pyspark.sql import functions as f
dfVarAndPredAgg = dfVarAndPred.groupBy("prediction").agg(\
                                      f.avg('contactos').alias('contactos'),\
                                    f.avg('mt_sumSegMobile').alias('mt_sumSegMobile'),\
                                    f.avg('mt_sumSegService').alias('mt_sumSegService'),\
                                    f.avg('mt_sumSegInternational').alias('mt_sumSegInternational'),\
                                    f.avg('mt_sumSegOthers').alias('mt_sumSegOthers'),\
                                    f.avg('mt_sumLlaMobile').alias('mt_sumLlaMobile'),\
                                    f.avg('mt_sumLlaService').alias('mt_sumLlaService'),\
                                    f.avg('mt_sumLlaInternational').alias('mt_sumLlaInternational'),\
                                    f.avg('mt_sumLlaOthers').alias('mt_sumLlaOthers'),\
                                    f.avg('mt_sumSmsMobile').alias('mt_sumSmsMobile'),\
                                    f.avg('mt_sumSmsInternational').alias('mt_sumSmsInternational'),\
                                    f.avg('mt_sumSmsOthers').alias('mt_sumSmsOthers'),\
                                    f.avg('mt_nroDestMobile').alias('mt_nroDestMobile'),\
                                    f.avg('mt_nroDestService').alias('mt_nroDestService'),\
                                    f.avg('mt_nroDestInternational').alias('mt_nroDestInternational'),\
                                    f.avg('mt_nroDestOthers').alias('mt_nroDestOthers'),\
                                    f.avg('mt_PropSmsMobile').alias('mt_PropSmsMobile'),\
                                    f.avg('mt_PropSmsInternational').alias('mt_PropSmsInternational'),\
                                    f.avg('mt_PropNroDestInternational').alias('mt_PropNroDestInternational'),\
                                    f.avg('mo_sumSegMobile').alias('mo_sumSegMobile'),\
                                    f.avg('mo_sumSegService').alias('mo_sumSegService'),\
                                    f.avg('mo_sumSegInternational').alias('mo_sumSegInternational'),\
                                    f.avg('mo_sumSegOthers').alias('mo_sumSegOthers'),\
                                    f.avg('mo_sumLlaMobile').alias('mo_sumLlaMobile'),\
                                    f.avg('mo_sumLlaService').alias('mo_sumLlaService'),\
                                    f.avg('mo_sumLlaInternational').alias('mo_sumLlaInternational'),\
                                    f.avg('mo_sumLlaOthers').alias('mo_sumLlaOthers'),\
                                    f.avg('mo_nroDestMobile').alias('mo_nroDestMobile'),\
                                    f.avg('mo_nroDestService').alias('mo_nroDestService'),\
                                    f.avg('mo_nroDestInternational').alias('mo_nroDestInternational'),\
                                    f.avg('mo_nroDestOthers').alias('mo_nroDestOthers'),\
                                    f.avg('PropSegMobileTvsTotal').alias('PropSegMobileTvsTotal'),\
                                    f.avg('PropContactosTvsTotal').alias('PropContactosTvsTotal'))

In [46]:
dfVarAndPredAgg.limit(10).toPandas().head()

,prediction,contactos,mt_sumSegMobile,mt_sumSegService,mt_sumSegInternational,mt_sumSegOthers,mt_sumLlaMobile,mt_sumLlaService,mt_sumLlaInternational,mt_sumLlaOthers,...,mo_sumLlaMobile,mo_sumLlaService,mo_sumLlaInternational,mo_sumLlaOthers,mo_nroDestMobile,mo_nroDestService,mo_nroDestInternational,mo_nroDestOthers,PropSegMobileTvsTotal,PropContactosTvsTotal
0,1,3.045636,268.382147,0.000082,1.414738,7.123261,2.250149,0.000003,0.009526,0.055380,...,1.392671,0.016117,0.010077,0.023921,0.882511,0.015294,0.007385,0.019197,0.669105,0.715911
1,3,5.067388,247.191611,0.000052,0.889074,2.515011,2.193726,0.000002,0.006337,0.021131,...,3.866884,2.415908,0.030456,0.041797,2.009769,1.158814,0.019066,0.029788,0.267450,0.571031
2,4,8.076050,509.063534,0.002783,476.937793,20.146293,4.569193,0.000092,1.680540,0.144487,...,4.967689,0.445239,2.045490,0.173646,2.566189,0.221477,0.927084,0.115051,0.589941,0.648598
3,2,6.927536,708.768116,369.971014,180.695652,17.623188,4.739130,1.275362,1.289855,0.333333,...,3.173913,0.188406,0.623188,0.043478,1.739130,0.130435,0.275362,0.043478,0.873099,0.796217
4,0,11.588789,1024.673578,0.000000,2.425931,41.128691,8.984218,0.000000,0.016739,0.302664,...,12.250825,0.260203,0.019959,0.302884,5.817629,0.169048,0.013905,0.188290,0.469821,0.672314


In [48]:
dfVarAndPredAgg \
  .coalesce(1) \
  .write \
  .mode('overwrite') \
  .option('header', 'true') \
.option('sep','|')\
  .csv('../prepData/output.csv')

In [49]:
dfVarAndPredAgg.limit(10).toPandas().to_csv('../prepData/output2.csv', sep='|',index=False)

In [50]:
dfVarAndPred.agg(\
                                      f.avg('contactos').alias('contactos'),\
                                    f.avg('mt_sumSegMobile').alias('mt_sumSegMobile'),\
                                    f.avg('mt_sumSegService').alias('mt_sumSegService'),\
                                    f.avg('mt_sumSegInternational').alias('mt_sumSegInternational'),\
                                    f.avg('mt_sumSegOthers').alias('mt_sumSegOthers'),\
                                    f.avg('mt_sumLlaMobile').alias('mt_sumLlaMobile'),\
                                    f.avg('mt_sumLlaService').alias('mt_sumLlaService'),\
                                    f.avg('mt_sumLlaInternational').alias('mt_sumLlaInternational'),\
                                    f.avg('mt_sumLlaOthers').alias('mt_sumLlaOthers'),\
                                    f.avg('mt_sumSmsMobile').alias('mt_sumSmsMobile'),\
                                    f.avg('mt_sumSmsInternational').alias('mt_sumSmsInternational'),\
                                    f.avg('mt_sumSmsOthers').alias('mt_sumSmsOthers'),\
                                    f.avg('mt_nroDestMobile').alias('mt_nroDestMobile'),\
                                    f.avg('mt_nroDestService').alias('mt_nroDestService'),\
                                    f.avg('mt_nroDestInternational').alias('mt_nroDestInternational'),\
                                    f.avg('mt_nroDestOthers').alias('mt_nroDestOthers'),\
                                    f.avg('mt_PropSmsMobile').alias('mt_PropSmsMobile'),\
                                    f.avg('mt_PropSmsInternational').alias('mt_PropSmsInternational'),\
                                    f.avg('mt_PropNroDestInternational').alias('mt_PropNroDestInternational'),\
                                    f.avg('mo_sumSegMobile').alias('mo_sumSegMobile'),\
                                    f.avg('mo_sumSegService').alias('mo_sumSegService'),\
                                    f.avg('mo_sumSegInternational').alias('mo_sumSegInternational'),\
                                    f.avg('mo_sumSegOthers').alias('mo_sumSegOthers'),\
                                    f.avg('mo_sumLlaMobile').alias('mo_sumLlaMobile'),\
                                    f.avg('mo_sumLlaService').alias('mo_sumLlaService'),\
                                    f.avg('mo_sumLlaInternational').alias('mo_sumLlaInternational'),\
                                    f.avg('mo_sumLlaOthers').alias('mo_sumLlaOthers'),\
                                    f.avg('mo_nroDestMobile').alias('mo_nroDestMobile'),\
                                    f.avg('mo_nroDestService').alias('mo_nroDestService'),\
                                    f.avg('mo_nroDestInternational').alias('mo_nroDestInternational'),\
                                    f.avg('mo_nroDestOthers').alias('mo_nroDestOthers'),\
                                    f.avg('PropSegMobileTvsTotal').alias('PropSegMobileTvsTotal'),\
                                    f.avg('PropContactosTvsTotal').alias('PropContactosTvsTotal')).limit(10).toPandas().to_csv('../prepData/output3.csv', sep='|',index=False)

In [55]:
dfVarAndPred.groupBy("prediction").count().show()

+----------+-------+
|prediction|  count|
+----------+-------+
|         1|2581009|
|         3| 477545|
|         4|  54254|
|         2|     69|
|         0| 648835|
+----------+-------+

